In [2]:
import cellbell
import numpy as np
import pandas as pd
import seaborn as sns
import json
import matplotlib.pyplot as plt
import psycopg2
from bs4 import BeautifulSoup
import requests
import re
from sklearn.ensemble import RandomForestRegressor
from constraint import *


/anaconda2/lib/python2.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


ImportError: No module named constraint

In [2]:
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 100)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
with open('config.json') as f:
    conf = json.load(f)

# Data Collection & Merging

In [ ]:
#Trying the database route
conn_str = "host={} dbname={} user={} password={}".format(conf['host'],conf['database'], conf['user'], conf['passw'])
conn = psycopg2.connect(conn_str)

In [ ]:
plays = pd.read_sql('select * from play_player ', con=conn)
games = pd.read_sql('select * from game ', con=conn)
players = pd.read_sql('select * from player ', con=conn)


In [ ]:
teams = pd.read_sql('select * from team',con=conn)
teams.to_csv('/Users/ianbury/Springboard/FantasyFootball/teams.csv')

In [3]:
teams = pd.read_csv('/Users/ianbury/Springboard/FantasyFootball/teams.csv')
teams.drop('Unnamed: 0',axis=1,inplace=True)

In [ ]:
#dropping non-grouping columns + defensive fields. Might create a defensive scoring dataframe later.
def_cols = []
for d in plays.columns.values.tolist():
    if d.startswith('defense'):
        def_cols.append(d)

def_cols.extend(('drive_id','play_id'))
plays_dropped = plays.drop(columns=def_cols)


In [ ]:
#Aggregating stats to game-player level
plays_sum = plays_dropped.groupby(by=['gsis_id','player_id','team']).sum().reset_index()

In [ ]:
#fixing inconsistent complete of name columns
def fix_names(row):
    """This function looks up the chain of various name columns to fix null column issues"""
    if not row['gsis_name']:
        if not row['full_name']:
            if not row['last_name']:
                    return None
            else:
                if not row['first_name']:
                    return row['first_name'][0]+'.'+row['last_name']
                else:
                    return row['last_name']
        else:
            return row['full_name']
    else:
        return row['gsis_name']

In [ ]:
#Applying fix_names function from above to create a new column
players.loc[:,'name_fixed'] = players.apply(fix_names,axis=1)

In [ ]:
#dropping extra name columns and other unneccessary columns now
players_dropped = players[['name_fixed','player_id','position','years_pro','height','weight']]
#bring back previous school? Don't drop too early. Do some statistical digging first.

In [ ]:
#joining players to aggregated plays
pp = plays_sum.merge(players_dropped, on='player_id', how='left')

In [ ]:
#dropping unnecessary columns from games
games_dropped = games[['gsis_id','start_time','week','day_of_week','season_year','season_type','home_team','away_team','home_score','away_score']]

In [ ]:
#joining players-plays to games
ppg = pp.merge(games_dropped, on='gsis_id', how='left')

In [ ]:
#Export data as a checkpoint
#ppg.to_csv('/Users/ianbury/player-game-stats.csv')
%ding
#could use pickle

# Start here

In [3]:
#Loading from checkpoint
ppg = pd.read_csv('/Users/ianbury/Springboard/FantasyFootball/player-game-stats.csv', parse_dates=['start_time'])
teams = pd.read_csv('/Users/ianbury/Springboard/FantasyFootball/teams.csv')
teams.drop('Unnamed: 0',axis=1,inplace=True)
ppg.head()
#could use min-max scaler on height/weight/year_pro (important for K-mean)

,Unnamed: 0,gsis_id,player_id,team,fumbles_forced,fumbles_lost,fumbles_notforced,fumbles_oob,fumbles_rec,fumbles_rec_tds,...,weight,start_time,week,day_of_week,season_year,season_type,home_team,away_team,home_score,away_score
0,0,2009080950,00-0003292,BUF,0,0,0,0,0,0,...,247.0,2009-08-10,0,Sunday,2009,Preseason,TEN,BUF,21,18
1,1,2009080950,00-0009056,BUF,0,0,0,0,0,0,...,265.0,2009-08-10,0,Sunday,2009,Preseason,TEN,BUF,21,18
2,2,2009080950,00-0011626,BUF,0,0,0,0,0,0,...,174.0,2009-08-10,0,Sunday,2009,Preseason,TEN,BUF,21,18
3,3,2009080950,00-0012478,BUF,0,0,0,0,0,0,...,224.0,2009-08-10,0,Sunday,2009,Preseason,TEN,BUF,21,18
4,4,2009080950,00-0019310,BUF,0,0,0,0,0,0,...,227.0,2009-08-10,0,Sunday,2009,Preseason,TEN,BUF,21,18


# Translating Stats to Fantasy Points

In [5]:
#Might add a config file here to set your league's settings in the future


In [4]:
#adding list of relevant columns to scoring
scoring_cols = ['fantasy_score',
                'fumbles_rec_tds',
                'fumbles_lost',
                'passing_int',
                'passing_yds',
                'passing_tds',
                'passing_twoptm',
                'receiving_rec',
                'receiving_tds',
                'receiving_twoptm',
                'receiving_yds',
                'rushing_yds',
                'rushing_tds',
                'rushing_twoptm',
                'kicking_rec_tds',
                'puntret_tds',
                'receiving_tar',
                'rushing_att',
                'passing_att'
                    ]

In [5]:
#List of relevant non-scoring columns
feature_cols=['gsis_id',
 'player_id',
 'team',
 'name_fixed',
 'position',
 'years_pro',
 'height',
 'weight',
 'start_time',
 'week',
 'day_of_week',
 'season_year',
 'season_type',
 'home_team',
 'away_team',
 'home_score',
 'away_score']
#add back in some of the dropped features from previous cells.

In [6]:
def scoring(row):
    """This function take all the columns relevant for scoring and apply their point values, 
    resulting in a fantasy score for each player-game"""
    passing = row['passing_int']*(-1)+row['passing_yds']*0.04+row['passing_tds']*4+row['passing_twoptm']*2
    rushing = row['rushing_yds']*0.1+row['rushing_twoptm']*2+row['rushing_tds']*6
    receiving = row['receiving_twoptm']*2+row['receiving_tds']*6+row['receiving_yds']*0.1+row['receiving_rec']*0.5
    fumbles = row['fumbles_rec_tds']*6+row['fumbles_lost']*(-2)
    returns = row['kicking_rec_tds']*6+row['puntret_tds']*6
   
    score = passing+rushing+receiving+fumbles+returns
    

    return score
#could leave per-category (per stat line) predictions
#could produce confidence interval as well

In [7]:
#Applying scoring
ppg['fantasy_score']=ppg.apply(scoring,axis=1)

In [8]:
#dropping unnecessary columns
ppg = ppg[feature_cols+scoring_cols]


In [9]:
#We already have day of week and week of schedule, so let's truncate start_time to only be game time HH:MM for early vs. late game as a feature
ppg[['start_time', 'home_team','away_team']].sample(n=15)
#You'd think the offset would match the home_team timezone but that doesn't look to be the case.

,start_time,home_team,away_team
133026,2014-11-02 18:00:00,KC,NYJ
153335,2015-10-11 20:05:00,DET,ARI
118009,2014-01-11 21:35:00,SEA,NO
23647,2010-01-17 21:40:00,SD,NYJ
188923,2017-08-09 23:30:00,CAR,HOU
139530,2014-12-20 21:30:00,WAS,PHI
56646,2011-09-25 20:15:00,CHI,GB
125410,2014-09-07 20:25:00,DAL,SF
151205,2015-09-27 17:00:00,BAL,CIN
150779,2015-09-27 17:00:00,CAR,NO


In [10]:
#Let's see all the different positions we have in the set. 
ppg['position'].unique()

array(['UNK', 'QB', 'RB', 'DE', 'TE', 'DB', 'WR', 'DT', 'K', 'P', 'LB',
       'OT', 'OG', 'NT', 'C', 'LS', 'T', 'FB', 'CB', 'SAF', 'OLB', 'OL',
       'ILB'], dtype=object)

In [11]:
#Focusing on only offensive positions for now. Also merging FB & RB
ppg['position'].replace('FB','RB',inplace=True)
off_pos = ['QB','RB','TE','WR']


In [12]:
#Removing Preseason and Postseason games due to inconsistent week labels
ppg=ppg[ppg['season_type']=='Regular']

In [13]:
#Creating a dict of dataframes of Fantasy scores and features by game-player by postion
pos_scores={}
for o in off_pos:
    pos_scores[o] = ppg[ppg['position']==o]


In [14]:
#Alternatively, using multi-index
ppg_mi = ppg.set_index(['position','season_year','week']).sort_index()
mi_sorted = ppg_mi.loc[off_pos]

In [15]:
#Something's wrong with the original data around pre-season games and week number not being consistent year-to-year.
idx=pd.IndexSlice
ppg_mi.loc[idx[:,:,:],['home_team','away_team','season_type','start_time','gsis_id']].groupby(['season_year','week','season_type'])['gsis_id'].nunique()



season_year  week  season_type
2009         1     Regular        16
             2     Regular        16
             3     Regular        16
             4     Regular        14
             5     Regular        14
             6     Regular        14
             7     Regular        13
             8     Regular        13
             9     Regular        13
             10    Regular        15
             11    Regular        16
             12    Regular        16
             13    Regular        16
             14    Regular        16
             15    Regular        16
             16    Regular        16
             17    Regular        16
2010         1     Regular        16
             2     Regular        16
             3     Regular        16
             4     Regular        14
             5     Regular        14
             6     Regular        14
             7     Regular        14
             8     Regular        13
             9     Regular        13
       

In [16]:
#Due to inconsistent week labelling, I'm going to filter our all Postseason & Preseason games.
ppg_mi = ppg_mi[ppg_mi['season_type']=='Regular']

# Web scraping - Weather Data

In [ ]:
#example_url= 'http://www.nflweather.com/en/week/2014/week-1/'


In [ ]:
#Merge on week-year-hometeam

In [ ]:
url_base = 'http://www.nflweather.com/en/week/'
week=''
year=''
soup_mix=[]
for year in range(2009,2018):
    for week in range(1,18):
        r=requests.get(url_base+str(year)+'/week-'+str(week))
        html_doc = r.text
        soup = BeautifulSoup(html_doc, 'html.parser')
        df = parse_forecast(soup)
        df['Year']=year
        df['Week']=week
        soup_mix.append(df)

In [ ]:
weather = pd.concat(soup_mix)
#checkpoint for weather data, so I don't have to spam the weather url
weather.to_csv('/Users/ianbury/Springboard/FantasyFootball/weather.csv')

In [ ]:
#Forecasts start on element 9, then every 13 elements.
def parse_forecast(soup):
    """Takes the html output of a BeautifulSoup object, 
    parses out elements from 5th and 9th columns (home team and weather forecast in this case), and returns a dataframe."""
    d={}
    f=[]
    home=[]
    for k,v in enumerate(soup.find_all('td')):
        if k in range(5,500,13):
            home.append(v.text.strip())
        if k in range(9,500,13):
            f.append(v.string.strip())
    d['home']=home
    d['forecast']=f
    df=pd.DataFrame(d)
    return df

In [17]:
#Loading from checkpoint
weather=pd.read_csv('/Users/ianbury/Springboard/FantasyFootball/weather.csv', index_col=False)

In [18]:
#dropping St.Louis Rams & San Diego Charges - teams have moved locations
teams=teams.drop([22,25],axis=0)

In [19]:
#Merging weather with team names so we can merge with the mega dataframe
wt = weather.drop('Unnamed: 0',axis=1).merge(teams,how='left', left_on='home', right_on='name',validate='m:1')

In [20]:
#Let's classify the forecasts!
wt.forecast.nunique()
#A ton of unique values - time for regex


565

In [21]:
#Splitting forecast column into temperature and description
regex = re.compile(r'\d+f ')

wt['temp']= wt['forecast'].apply(lambda x: pd.to_numeric(x.split('f')[0].split('/')[0]) if bool(regex.search(x)) else np.nan)
wt['desc']=wt['forecast'].apply(lambda x: regex.split(x)[1] if bool(regex.search(x)) else x)


In [22]:
#A quick search shows most domes are temperature controlled at 65F.
wt.temp.fillna(65,inplace=True)

In [23]:
wt.desc.unique()
#There's probably a more elegant way to sort weather descriptions into categories - I used a spreadsheet.

array(['Mostly Cloudy', 'DOME', 'Fair', 'Partly Cloudy', 'Overcast',
       'A Few Clouds', 'Light Rain', 'Light Rain Fog/Mist', 'Fog/Mist',
       'Clear', 'Overcast with Haze', 'Rain Fog/Mist', 'Partly Sunny',
       'Overcast and Windy', 'Fog', 'Few Clouds', 'Snow',
       'Overcast and Breezy', 'A Few Clouds with Haze',
       'Mostly Cloudy and Breezy', 'Partly Cloudy with Haze',
       'Partly Cloudy and Breezy', 'Fair with Haze', 'Mostly Sunny',
       'Rain Showers', 'Light Drizzle', 'Sunny', 'Light Rain and Breezy',
       'Light Snow', ' Light Rain', 'Patchy Fog',
       '  Thunderstorm in Vicinity Light Rain Fog/Mist', 'NA',
       ' Light Drizzle', 'Fair and Breezy', 'Lt Rain',
       ' Rain Fog/Mist and Breezy', 'Rain', ' Light Rain Fog/Mist',
       ' Fog/Mist', 'Lt Rain, Fog', ' Light Snow Fog/Mist',
       ' Blowing Snow and Breezy', ' Thunderstorm Rain Fog/Mist',
       'Thunder, Lt Rain', 'Mostly Clear', 'Areas Drizzle', 'Breezy',
       ' Snow and Breezy', 'Areas Fog

In [24]:
wet = [' Light Rain','Light Rain Fog/Mist','Fog/Mist','Rain Fog/Mist','Fog','Rain Showers','Light Drizzle','Light Rain and Breezy','Light Rain','  Thunderstorm in Vicinity Light Rain Fog/Mist',' Light Drizzle','Lt Rain',' Rain Fog/Mist and Breezy','Rain',' Light Rain Fog/Mist',' Fog/Mist','Lt Rain, Fog',' Thunderstorm Rain Fog/Mist','Thunder, Lt Rain','Areas Drizzle','Showers',' Heavy Rain Fog/Mist',' Rain Fog/Mist',' Thunderstorm','  Showers in Vicinity',' Fog','Mod Rain, Fog','Drizzle','Foggy','Heavy Rain','Possible Drizzle']
good = ['Mostly Cloudy','DOME','Fair','Partly Cloudy','Overcast','A Few Clouds','Clear','Overcast with Haze','Partly Sunny','Few Clouds','A Few Clouds with Haze','Partly Cloudy with Haze','Fair with Haze','Mostly Sunny','Sunny','Patchy Fog','NA','Mostly Clear','Areas Fog','A few Clouds','Cloudy','Chance Showers','Dry and Partly Cloudy','Humid and Mostly Cloudy','Humid','Dry','Humid and Partly Cloudy','Possible Light Rain']
windy = ['Overcast and Windy','Breezy','Fair and Breezy','Mostly Cloudy and Breezy','Overcast and Breezy','Partly Cloudy and Breezy']
snow = [' Blowing Snow and Breezy',' Heavy Snow Freezing Fog',' Light Freezing Rain',' Light Snow',' Light Snow Fog/Mist',' Snow and Breezy','Flurries','Light Snow','Snow','Wintry Mix']

In [25]:
wt['desc_simple'] = wt.desc.apply(lambda x: 'Good' if x in good else 'Wet' if x in wet else 'Windy' if x in windy else 'Snowy' if x in snow else 'Missing')

In [26]:
#Merging into ppg master dataframe
ppg = ppg.merge(wt, how='left', left_on=['home_team','season_year','week'],right_on=['team_id','Year','Week'])

In [27]:
ppg[ppg['temp'].isnull()]['gsis_id'].nunique()/ppg['gsis_id'].nunique()
#missing weather data for 16% of the games


0

In [28]:
missing_weather={}
for y in ppg['season_year'].unique():
    missing_weather[y]=ppg[(ppg['temp'].isnull())&(ppg['season_year']==y)]['gsis_id'].nunique()/ppg[ppg['season_year']==y]['gsis_id'].nunique()

In [29]:
#Well that explains it - all of 2010 is missing. Confirmed via url that 2010 weather data 404s.
missing_weather


{2009: 0,
 2010: 1,
 2011: 0,
 2012: 0,
 2013: 0,
 2014: 0,
 2015: 0,
 2016: 0,
 2017: 0}

In [30]:
weekly_weather={}
for y in ppg['week'].unique():
    weekly_weather[y]=ppg[(ppg['temp'].isnull())&(ppg['week']==y)]['gsis_id'].nunique()/ppg[ppg['week']==y]['gsis_id'].nunique()

In [31]:
#No weekly trend on missing weather.
#Could be some PyLibs to find missing weather for a given geo.
weekly_weather

{1: 0,
 2: 0,
 3: 0,
 4: 0,
 5: 0,
 6: 0,
 7: 0,
 8: 0,
 9: 0,
 10: 0,
 11: 0,
 12: 0,
 13: 0,
 14: 0,
 15: 0,
 16: 0,
 17: 0}

# Adding in rolling averages

In [32]:
rolling=ppg.sort_values(['name_fixed','week']).reset_index(drop=True)

In [34]:
#Previous 3- and 5-week average scores
rolling['prev_score'] = rolling.groupby(['season_year','name_fixed'])['fantasy_score'].rolling(1).mean().shift().reset_index(level=[0,1])['fantasy_score']
rolling['avg3_score'] = rolling.groupby(['season_year','name_fixed'])['fantasy_score'].rolling(3).mean().shift().reset_index(level=[0,1])['fantasy_score']
rolling['avg5_score'] = rolling.groupby(['season_year','name_fixed'])['fantasy_score'].rolling(5).mean().shift().reset_index(level=[0,1])['fantasy_score']
rolling['prev_tar']=rolling.groupby(['season_year','name_fixed'])['receiving_tar'].rolling(1).mean().shift().reset_index(level=[0,1])['receiving_tar']
rolling['avg3_tar']=rolling.groupby(['season_year','name_fixed'])['receiving_tar'].rolling(3).mean().shift().reset_index(level=[0,1])['receiving_tar']
rolling['avg5_tar']=rolling.groupby(['season_year','name_fixed'])['receiving_tar'].rolling(5).mean().shift().reset_index(level=[0,1])['receiving_tar']
rolling['prev_rush_att']=rolling.groupby(['season_year','name_fixed'])['rushing_att'].rolling(1).mean().shift().reset_index(level=[0,1])['rushing_att']
rolling['avg3_rush_att']=rolling.groupby(['season_year','name_fixed'])['rushing_att'].rolling(3).mean().shift().reset_index(level=[0,1])['rushing_att']
rolling['avg5_rush_att']=rolling.groupby(['season_year','name_fixed'])['rushing_att'].rolling(5).mean().shift().reset_index(level=[0,1])['rushing_att']
rolling['prev_pass_att']=rolling.groupby(['season_year','name_fixed'])['passing_att'].rolling(1).mean().shift().reset_index(level=[0,1])['passing_att']
rolling['avg3_pass_att']=rolling.groupby(['season_year','name_fixed'])['passing_att'].rolling(3).mean().shift().reset_index(level=[0,1])['passing_att']
rolling['avg5_pass_att']=rolling.groupby(['season_year','name_fixed'])['passing_att'].rolling(5).mean().shift().reset_index(level=[0,1])['passing_att']
%ding

In [35]:
checkpt = rolling.to_csv('/Users/ianbury/Springboard/FantasyFootball/chckpt1.csv')

# FanDuel Data and Structure
Each team has a salary cap of $60,000 to fill 9 positions [1-QB, 2-RB, 3-WR, 1-TE, 1-FLEX, 1-DEF]. FLEX can be either a WR, RB or TE. Example salary data for preseason 2018 is below.

In [ ]:
fd = pd.read_csv('/Users/ianbury/Downloads/FanDuel-NFL-2018-09-09-27409-players-list.csv')

In [ ]:
fd.head()

In [ ]:
fd.groupby('Position')['Salary'].describe()

In [ ]:
fd=fd.set_index('Position').sort_index()

In [ ]:
problem = Problem()
problem.addVariable('RB1',fd.loc['RB'].Salary)
problem.addVariable('RB2',fd.loc['RB'].Salary)
problem.addVariable('QB',fd.loc['QB'].Salary)
#problem.addVariable('WR1',fd.loc['WR'].Salary)
#problem.addVariable('WR2',fd.loc['WR'].Salary)
#problem.addVariable('WR3',fd.loc['WR'].Salary)
#problem.addVariable('TE',fd.loc['TE'].Salary)
#problem.addVariable('FLEX',fd.loc[['TE','QB','RB']].Salary)
#problem.addVariable('D',fd.loc['D'].Salary)
problem.addConstraint(MaxSumConstraint(30000))
problem.addConstraint(MinSumConstraint(24000))

In [ ]:
a = problem.getSolutions()

In [ ]:
print('k')
%ding